In [15]:
import json
from newspaper import Article, fulltext

import spacy
from spacy import displacy

In [4]:
cnn_json = json.loads(open('cnn_urls.json').read())

In [6]:
cnn_json[0]

'https://www.cnn.com/interactive/2022/07/politics/steve-bannon-us-democracy/'

In [27]:
len(cnn_json)

32958

In [9]:
article = Article(cnn_json[0])

In [12]:
article.download()
article.parse()

In [13]:
print(article.title)
print(article.authors)
print(article.publish_date)
print(article.text)

Power in chaos: Steve Bannon is disrupting democracy. This is how.
[]
None
As a single mom with a full-time job, Wendy Ahrenkiel had little time for politics. But after the 2020 election, when Joe Biden was declared President, she felt cheated — and found the time.

Ahrenkiel, 47, has heard the arguments in the mainstream media about how claims of a stolen election in 2020 are unsupported by credible evidence. But she still doesn’t buy it.

“I thought that there's a lot of funny things that went on with the election,” she told CNN. “That made me just do my own research.”

That research led her to talk politics with friends — many of them moms like her — who clued her in to a show called “War Room,” hosted by a man with whom she was only vaguely familiar: Stephen K. Bannon.

Bannon doesn’t just touch on the election canard, he pounds it with a sledgehammer day after day on “War Room” — a program that reaches audiences online and on air.

The audio version is usually among the top three 

In [18]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB ? eta 0:00:000:00:0100:05
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
#!python -m spacy download en_core_web_sm

In [ ]:
#python -m spacy download en

In [19]:
# apply NER on article.text
nlp = spacy.load("en_core_web_lg")

In [20]:
displacy.render(nlp(article.text), style="ent", jupyter=True)

In [23]:
# get all person full names from article.text
full_names = set()

for ent in nlp(article.text).ents:
    if ent.label_ == 'PERSON':
        # check if ent is a full name
        if len(ent.text.split()) > 1:
            full_names.add(ent.text)


In [24]:
full_names

{'Alex Jones',
 'Andrew Breitbart',
 'Anthony Fauci',
 'Ben Harnwell',
 'Benjamin Ginsberg',
 'Bill Barr',
 'Bill Clinton',
 'Bill Gates',
 'Billy Bush',
 'Bob Woodward',
 'Boris Epshteyn',
 'Brad Parscale',
 'Brad Raffensperger',
 'Christopher Wray',
 'Dan Schultz',
 'Dan Schultz’s',
 'David Chalian',
 'Diego Morales',
 'Dion Heimink',
 'Donald Trump',
 'Doug Mastriano',
 'Drew Griffin',
 'Eric Greitens',
 'Franklin D. Roosevelt',
 'George Washington',
 'Hillary Clinton',
 'J.R. Majewski',
 'Jeff Sessions',
 'Jim Marchant',
 'Jim Waurishuk',
 'Jocelyn Benson',
 'Joe Biden',
 'John Eastman',
 'John McCain',
 'Joshua Green',
 'Kandiss Taylor',
 'Kristina Karamo',
 'Lou Dobbs',
 'Marjorie Taylor Greene',
 'Mark Finchem',
 'Matthew DePerno',
 'Merrick Garland',
 'Mike Pence',
 'Miles Taylor',
 'Mother Jones',
 'Nancy Pelosi',
 'Robert Costa',
 'Ron Filipkowski',
 'Ruby Freeman',
 'Rudy Giuliani',
 'Stacy Altiery',
 'Stephen K. Bannon',
 'Steve Bannon',
 'Tina Peters',
 'Tucker Carlson',
 

In [26]:
len(full_names)

57

In [28]:
# iterate over all cnn_json and get all full names
full_names = set()

for url in cnn_json:
    try:
        article = Article(url)
        article.download()
        article.parse()
        for ent in nlp(article.text).ents:
            if ent.label_ == 'PERSON':
                # check if ent is a full name
                if len(ent.text.split()) > 1:
                    full_names.add(ent.text)
    except:
        continue

In [29]:
# save full_names to a csv file
import csv

with open('cnn_full_names.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['full_name'])
    for full_name in full_names:
        writer.writerow([full_name])

In [1]:
!head -10 cnn_full_names.csv

full_name
Monika Skolimowska
Ryan Green/Universal Pictures
Nancy Nydam
Briton Macer Gifford
Walidah Imarisha
Ashley Settle
Edward Timbers
Berhala Reping
Bill Weir 00:05:16


In [2]:
!du -sh cnn_full_names.csv

1.1M	cnn_full_names.csv


In [3]:
!wc -l cnn_full_names.csv

69187 cnn_full_names.csv


In [9]:
import pandas as pd

df = pd.read_csv('cnn_full_names.csv')

In [10]:
df.head(10)

,full_name
0,Monika Skolimowska
1,Ryan Green/Universal Pictures
2,Nancy Nydam
3,Briton Macer Gifford
4,Walidah Imarisha
5,Ashley Settle
6,Edward Timbers
7,Berhala Reping
8,Bill Weir 00:05:16
9,Marc Anthony


In [11]:
df.shape

(67384, 1)

In [12]:
# Ryan Green/Universal Pictures
# find names that have "/"
df[df['full_name'].str.contains('/')]

,full_name
1,Ryan Green/Universal Pictures
16,Elva Etienne/Moment RF/Getty Images\n\nHow
27,Olivier Douliery/AFP/Getty Images
46,Hollie Adams/
52,Mat Zain/NurPhoto/Getty Images
...,...
67226,Gabriel Bouys/
67230,Frank Augstein/
67311,Hannah Mattix/
67332,Scott Olson/Getty Images\n\nStores


In [15]:
# replace "/.*" with "" in full names
df['full_name'] = df['full_name'].str.replace(r'/.*', '', regex=True)

In [17]:
df.head(10)

,full_name
0,Monika Skolimowska
1,Ryan Green
2,Nancy Nydam
3,Briton Macer Gifford
4,Walidah Imarisha
5,Ashley Settle
6,Edward Timbers
7,Berhala Reping
8,Bill Weir 00:05:16
9,Marc Anthony


In [18]:
df[df['full_name'].str.contains(r'\n')]

,full_name
16,Elva Etienne\n\nHow
130,Stephen Maturen\n\nPatriots
290,Pinocchio”\n\nBest
336,Kateryna\n\nKateryna's
598,Vera Bradley\n\n
...,...
67034,Mikhail Svetlov\n\nCalculations
67087,Moses Bwayo\n\n
67332,Scott Olson\n\nStores
67349,Courtesy Alisabeth Hayden\n\nHayden


In [21]:
df['full_name'] = df['full_name'].str.replace(r'\n.*', '', regex=True)

In [24]:
df.sample(n=10)

,full_name
34549,Fritz Von Erich
17143,Thomas Beauford Jr.
3040,Arif Kamal
41998,Alexander Korablyov
50669,Sune Bergstrom
58666,Victoire Ingabire Umuhoza Victoire Ingabire Um...
28703,Amelia Earhart
56446,Lola Pendande
36782,Kristi Yamaguchi
8290,Stephanie O’Grady


In [25]:
df.to_csv('cnn_full_names_clean.csv', index=False)